In [1]:
import os
from traitlets.config.manager import BaseJSONConfigManager
path = os.path.join(os.environ['HOME'], '.jupyter', 'nbconfig')
cm = BaseJSONConfigManager(config_dir=path)
_ = cm.update('livereveal', 
              {'width': 1024,
               'height': 768,
               #'width': 1000,
               #'height': 1200,
               'theme': 'white',
               'transition': 'linear',
               'display': 'block',
               'fragments': 'true',
               'minScale': 1.2,
               })

In [2]:
###### general and notebook related imports and settings
import numpy as np
from IPython.display import Markdown, HTML, display
# no warnings
import warnings
warnings.filterwarnings("ignore")
# markdown to html
from mistune import markdown

do_demo = False

if do_demo:
    # IPython display and widgets
    from IPython.display import HTML, Markdown
    from IPython.display import clear_output, display
    from ipywidgets import fixed, interact_manual
    # plotly with pandas support
    import plotly
    import plotly.graph_objs as ply_go
    import cufflinks as cf
    plotly.offline.init_notebook_mode(connected=True)
    # custom plotly extension
    # based on https://github.com/hyliu1989/PlotlyOfflineStream
    from plotlyStream import JupyterNotebookPlotlyStream

    ###### application related imports
    import asyncio
    from collections import OrderedDict
    # local
    import oxy
    import mqtt
    from mqtt import run_subscription, loggera

In [3]:
###### setup async process
###### see also http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Asynchronous.html 

from ipykernel.eventloops import register_integration

@register_integration('asyncio')
def loop_asyncio(kernel):
    '''Start a kernel with asyncio event loop support.'''
    loop = asyncio.get_event_loop()

    def kernel_handler():
        loop.call_soon(kernel.do_one_iteration)
        loop.call_later(kernel._poll_interval, kernel_handler)

    loop.call_soon(kernel_handler)
    try:
        if not loop.is_running():
            loop.run_forever()
    finally:
        loop.run_until_complete(loop.shutdown_asyncgens())
        loop.close()

In [4]:
def get_oo_init(oo_prev=None):
    # plotly does not work with empty data
    oo = oxy.Oxy()
    if oo_prev is not None:
         oo.data = oo_prev.data.median().to_frame().T
    return oo

###### function for notebook interaction
def fig_clear(fig):
    # matplotlib
    for ax in fig.axes:
        ax.cla()

def _plotly_get_fig(df, x, y, lcolors):
    # kwargs ...
    return df.iplot(
        x=x, y=y, colors=lcolors,
        kind='scatter', mode='lines+markers', 
        symbol='o', size=4,
        asFigure=True,
        theme='white',
        subplots=True,
        shape=(len(y),1),
        shared_xaxes=True,
        #layout=ply_layout,
    )
        
def pstream_clear(pstream):
    # plotly
    if hasattr(pstream, '_oo'):
        oo_prev = pstream._oo
    else:
        oo_prev = None
    oo = get_oo_init(oo_prev=oo_prev)
    #print(oo.data)
    pstream._fig = plotly_get_fig(oo)
    pstream._oo = oo
    
def pstream_do_clear():
    pstream_clear(pstream)
    pstream.update()
    
if do_demo:
    %gui asyncio
    loop = asyncio.get_event_loop()

    #debug = True
    debug = False

    if not debug:
        logger.propagate = False

    ## loop count
    #n = 10
    ## ... infinite
    n = None

    leds_colors = OrderedDict(zip(['IR', 'RED'], ['#A38C89', '#C82C1C']))
    leds = list(leds_colors.keys())
    lcolors = list(leds_colors.values())

    leds_ac = [led + '_ac' for led in leds]

    oo_init = get_oo_init()
    #oo_init.data
    
    def plotly_get_fig(oo, 
                   leds=leds, 
                   lcolors=lcolors):
        df = oo.data
        # create 2 axis layout with partial led data
        _ply_fig = _plotly_get_fig(df, 'time_sec', 
                                   leds[:1] + leds_ac[:1],
                                   lcolors)
        ply_layout = _ply_fig['layout']
        # create figure ...
        ply_fig = _plotly_get_fig(df, 'time_sec', 
                                  leds + leds_ac,
                                  lcolors * 2)
        # ... and fix axes and layout
        ply_fig['data'][1]['yaxis'] = 'y1'
        ply_fig['data'][2]['yaxis'] = 'y2'
        ply_fig['data'][3]['yaxis'] = 'y2'
        ply_fig['layout'] = ply_layout
        #ply_fig['layout'].update(ply_layout)
        return ply_fig

In [5]:
if do_demo:

    md1 = """
    ##### test
    $I = I_0 \exp(-\mu_s x)$
    """
    md2 = """
    ##### test2
    $E = m c^2$
    """

    import itertools
    md_texts = itertools.cycle([md1, md2])

    def get_md():
        return md_texts.__next__()

    def html_join(html, colR):
        return col2_html(html, colR, idR='graph_table_R', flexR=0.3, modeL='html', as_raw=True)

    from functools import partial
    get_html_join = partial(html_join, colR=get_md())

    ###### widgets creation
    ## init plotly stream instance
    pstream = JupyterNotebookPlotlyStream()
    pstream_clear(pstream)
    pstream.setToPlotInNewCell()

    html_tmpl = """
    <div class="alert alert-block alert-warning">
    <right>
    {}
    </right>
    </div>
    """

    #pstream.firstRun(opt_html_table_side=html_tmpl.format(html1))
    def pdisplay():
        from htmldom import htmldom
        dom = htmldom.HtmlDom()
        dom.parseHTML( pstream._html )
        elem = dom.find('div[id=graph_table_R]')
        new = 'test'+elem.text()
        elem.text(get_md())
        pstream._html = dom.find('*').html()
        #pstream.show()


    #funcw = interact_manual(fig_clear, fig=fixed(fig))
    funcw = interact_manual(pstream_do_clear)
    funcw.widget.children[0].description = 'clear plots'
    funcd = interact_manual(pdisplay)
    funcd.widget.children[0].description = '>'

    #pstream.firstRun(html_join=get_html_join, show=True)
    pstream.show()

    from htmldom import htmldom
    dom = htmldom.HtmlDom()
    dom.parseHTML( pstream._html )
    elem = dom.find('div[id=graph_table_R]')
    #new = 'test'+elem.text()
    #elem.text(new)
    #dom.find('*').html()
    elem.text()

    pstream._html
    _plot_html, plotdivid, _, _ = pstream._plot_html()
    #pstream._div_id, plotdivid, _plot_html.split(str(plotdivid))
    pstream._update(1024)
    #pstream._obj.data

    def consumer_action(buffer, pstream=pstream, keep=oo_init):
        # take new data
        oo = oxy.Oxy(buffer, do_eval=False)
        # update evaluation
        keep.eval()
        # update plot from keep
        pstream._fig = plotly_get_fig(keep)
        pstream._oo = keep
        pstream.update()
        #
        return oo

    _ = asyncio.ensure_future(run_subscription(consumer_action=consumer_action, 
                                               produce_n=n, 
                                               consume_n=n,
                                               consume_keep=oo_init))

    #pstream._oo.data
    #pstream._fig

In [6]:
###### plotly layout
# ply_layout = ply_go.Layout(
#     autosize=False,
#     width=800,
#     height=600,
#     margin=ply_go.Margin(
#         l=50,
#         r=50,
#         b=100,
#         t=100,
#         pad=4
#     ),
#     paper_bgcolor='#f4eded',
#     plot_bgcolor='#ffffff'
# )

In [7]:
%%HTML
<style>
.box{
    display: flex;
    flex-direction: row;
    flex-wrap: nowrap;
    justify-content: flex-start;
    align-content: stretch;
    width: 100%;
    height: 70%;
    margin: auto;
    margin-bottom: 2em;
    /*
    background-color: gray;
    */
}
.col{
    flex: 1;
}
</style>

In [8]:
md_kw = dict(parse_inline_html=True,
             escape=False)


In [9]:
def col2_html(colL, colR, 
              flexL=1, flexR=1,
              idL=None, idR=None,
              header='',
              modeL=None, modeR=None, modeH=None,
              mode_default='markdown', 
              as_raw=False):
    if modeL is None:
        modeL = mode_default
    if modeR is None:
        modeR = mode_default
    if modeH is None:
        modeH = mode_default
    if modeL == 'markdown':
        colL = markdown(colL, **md_kw)
    if modeR == 'markdown':
        colR = markdown(colR, **md_kw)
    if modeL == 'markdown':
        header = markdown(header, **md_kw)
    if header:
        header = """
        {}
        """.format(header)
    if idL is not None:
        idL = 'id="{}"'.format(idL)
    if idR is not None:
        idR = 'id="{}"'.format(idR)
    raw = """
{h}
<div class="box">
<div {id1} class="col" style="flex: {f1}">
{c1}
</div>
<div {id2} class="col" style="flex: {f2}">
{c2}
</div>
</div>
    """.format(c1=colL, c2=colR, 
               f1=flexL, f2=flexR, 
               id1=idL, id2=idR,
               h=header)
    if as_raw:
        return raw
    else:
        return HTML(raw)


$\LaTeX$ ...

$\require{mhchem}$
$\require{AMSmath}$
$\require{AMSsymbols}$
$\require{autobold}$

$\newcommand{\nicefrac}[2]{\left. #1\right/#2}$

$\newcommand{\conc}[1]{\left[#1\right]}$

$\newcommand{\hb}{\sf{Hb}}$
$\newcommand{\hbco}{\sf{HbCO}}$
$\newcommand{\Iac}{I_{AC}}$
$\newcommand{\Idc}{I_{DC}}$
$\newcommand{\hbm}{\sf{MetHb}}$
$\newcommand{\hbo}{\sf{HbO_2}}$
$\newcommand{\mux}[2]{\mu_a^{#1}\left(\lambda_{#2}\right)}$
$\newcommand{\oo}{\ce{O2}}$
$\newcommand{\po}{\sf{pO_2}}$
$\newcommand{\pao}{\sf{PaO_2}}$
$\newcommand{\ros}{\sf{R_{OS}}}$
$\newcommand{\sao}{\sf{Sa}_{O_2}}$
$\newcommand{\spo}{\sf{Sp}_{O_2}}$

[//]: # ($\nicefrac{1}{2}$)
[//]: # ($\spo$)


In [10]:
colR1 = """
<img src="figs/Beuth.png"   style="width: 80%" />
"""
Markdown(colR1)


<img src="figs/Beuth.png"   style="width: 80%" />


In [11]:
colL1 = """
### Probevorlesung
#### Ralf Ahlbrink
"""
Markdown(colL1)


### Probevorlesung
#### Ralf Ahlbrink


In [12]:
header = """
# Pulsoxymeter

***
"""
Markdown(header)


# Pulsoxymeter

***


# Pulsoxymeter

***

In [13]:
display(col2_html(colL1, colR1, flexR=1.5))

##  Was ist ein Pulsoxymeter und wofür wird es genutzt?
***
### Definitionen und Anwendung


####  "Das Gerät" 
#### für Messung Herzschlag und $\oo$-Sättigung des Blutes


<img src="figs/hardware/Fingerpulsoximeter-AVAX_AV-50DL.png"  style="width: 70%" />


#### physiologische Bedeutung des  Parameters Sauerstoffsättigung

- Sauerstoffsättigung in Arterien zeigt Atemfunktion und Transportvermögen im Blut an

  - ca. 98% des Sauerstofftransportes über Hämoglobine in Erythrozyten
  
  - $1\,\oo$ als Ligand pro Hämgruppe, <br>
  4 Hämgruppen pro $\hb$, <br>
  Bindung erfolgt kooperativ
  
  
- der Sauerstoffpartialdruck charakterisiert den <br>
$\oo$-Austausch über Membranen

[//]: # (  - Lunge $\rightarrow$ Blutkreislauf )
  
[//]: # (  - Kapillaren $\rightarrow$ Gewebe )

<img src="figs/basics/KanI-Fig3.19-oxygen_saturation_S_over_partial_pressure_pO2.png" style="width: 50%" />

- Hypoxie: geringer arterieller $\oo$-Partialdruck
- **Hypoxygenation**: geringe Sauerstoffsättigung im arteriellen Blut ($\sao$)

##### die *funktionelle* Sauerstoffsättigung $\spo$

<div class="alert alert-block alert-info">
$$
\spo = \frac{\large\conc{\hbo}}{\large\conc{\hbo} + \conc{\hb}} \times 100\%
$$
</div class="alert alert-block alert-info">

- $\hb$ ist deoxygeniertes Hämoglobin
- $\hbo$ ist oxygeniertes Hämoglobin 

<div class="alert alert-block alert-info">
#### Pulsoxymeter
Instrument zur nichtinvasiven Bestimmung 

  der Sauerstoffsättigung von Hämoglobin im arteriellen Blut 
  
  durch Messung der Lichtabsorption
  
  ***
  
  Das Verfahren wird *Pulsoymetrie* genannt.
</div class="alert alert-block alert-info">


#### Optische Absorptionsmessung von Hämoglobin

konventionelles Verfahren: 2 LEDs, also "2 Wellenlängen", im roten und NIR Spektralbereich <br>
  $\rightarrow$ Unterscheidung von 2 Komponenten der *funktionellen* Sauerstoffsättigung in Hämoglobin

<img src="figs/basics/KanII-Fig5.1-principle.png"  style="width: 70%" />


<div class="alert alert-block alert-success">
### Kennen Sie Anwendungsbereiche
### von Pulsoxymetern?
</div class="alert alert-block alert-success">


- Erkennung von ...

[//]: # ( Schlafapnoe, Höhenkrankheit )

- Monitoring in  ... 

[//]: # ( Rettungsdienst, Intensivstation, Anästhesie )

- Neugeborene ...

[//]: # ( krit. angeborener Herzfehler )

[//]: # ( 'Wearables': Fitness )

Zur Bestimmung von $\sao$ inklusive *disfunktionaler* Hämoglobine, $\conc{\hbco}$ und $\conc{\hbm}$, messen Laborgeräte erweiterte Wellenlängenbereiche (z.B. "mehr als sieben unterschiedliche Wellenlängen") und nutzen eine entsprechend optimierte Auswertung 

<img src="figs/hardware/lab_devices-Masimo.png"  style="width: 35%" />

##### die "fraktionelle" Sauerstoffsättigung

$\sao = \frac{\large\conc{\hbo}}{\large\conc{\hbo} + \conc{\hb} + \conc{\hbco} + \conc{\hbm}} \times 100\%$

- Berücksichtig Anteile dysfunktioneller Hb-Derivate
  - $\hbco$: Kohlenmonoxid als Ligand
  - $\hbm$: Methämoglobin mit oxidiertem Häm-Eisen

<img src="figs/basics/rainbow_graph.png"  style="width: 50%" />

<div class="alert alert-block alert-warning">
###### Annahmen und Näherungen in der Praxis:
- Bei der Pulsoxymetrie wird die arterielle Sauerstoffsättigung bestimmt, aber nicht direkt der $\oo$-Partialdruck
- Messungen mit 2 Wellenlängen
  * $\spo$ ist dann *funktionelle* Sättigung
  * ... welche unter Ausschluss von kritischen Fällen (z.B. Verdacht auf Rauchvergiftung) der kompletten Sättigung gleichgesetzt wird: $\sao \approx \spo$
</div class="alert alert-block alert-warning">

#### Vertiefende Literatur: Medizinische Grundlagen und Anwendungen
1. Rüdiger Kramme - *Medizintechnik*
1. https://de.wikipedia.org/wiki/Hämoglobin

## Wie bestimmt man die Sauerstoffsättigung photometrisch?

***
### Grundlagen

<div class="alert alert-block alert-success">

### Welche Gesetzmäßigkeit wird bei 
### Messungen der Lichtabsorption
### standardmäßig verwendet?

</div class="alert alert-block alert-success">


### Lambert-Beer'sches Gesetz: $I(A) = I_0 10^{-A}$

$
\begin{align}
A(\lambda) &= \sum{\epsilon_i(\lambda) c_i x} \\
x &: \textsf{optische Weglänge} \\
\epsilon_i(\lambda) &: {\Tiny\textsf{molarer dekadischer }} \textsf{Extinktionskoeffizient } 
{\Tiny\textsf{für Stoff } i } \\
c_i &: \textsf{Konzentration } {\Tiny\textsf{Stoff } i } \\
\end{align}
$

***

Beispiele ermittelt mittels hämolysierter Blutproben (S. Prahl)

$
\begin{align}
\epsilon_{\hb}(660\;\sf{nm}) & \approx & 3227\;\sf{cm}^{-1}\sf{M}^{-1} \\
\epsilon_{\hbo}(660\;\sf{nm}) & \approx & 320\;\sf{cm}^{-1}\sf{M}^{-1}
\end{align}
$

###### Eine Formulierung für "Schichtstrukturen"

- $\epsilon_ic \rightarrow \mu^i_a$ 
- Schichtdicken $x_i$ anstelle einer Weglänge $x$:

$
\begin{aligned}
I  &= 
I_0 
\cdot  e^{-\color{blue}{\mu^A_a} \color{red}{x_A}} 
\cdot \textsf{...}
\qquad \textsf{(A: Arterien)} \\
 \color{blue}{\mu^A_a}\left(\color{green}{\spo}\right) &= \left(\color{green}{\spo}\color{blue}{\mu_a^\hbo} + (1 - \color{green}{\spo}) \color{blue}{\mu_a^\hb} \right ) 
\end{aligned}
$

***

Beispiel $\Tiny\textsf{(Vollblut mit Hämatokrit-Wert Hct 45%, 
N. Bosschaart et al 2013)}$:

$
\begin{align}
\mu_a^{\hb}(660\;\sf{nm}) & \approx & 1,64\;\sf{mm}^{-1} \\
\mu_a^{\hbo}(660\;\sf{nm}) & \approx & 0,15\;\sf{mm}^{-1}
\end{align}
$

<img src="figs/basics/KanII-Fig5.14-intensity_parts.png"  style="width: 70%" />

In [14]:
colL3 = """
<img src="figs/basics/Hb_and_LEDs_spectra.png"  style="width: 100%" />
"""
#Markdown(colL3)

In [15]:
colR3 = """
<img src="figs/eval/eval_oxy-steps_1to3-excerpt.png"  style="width: 100%" />
"""
#Markdown(colR3)

### Ein Messergebnis

In [16]:
display(col2_html(colL3, colR3))

#### Der "Trick": 
######  Die Pulsmodulation ergibt die Arterien-Schicht

- Pulsdruckwelle verändert Durchmesser der Arterien

  $\rightarrow $ mit Herzschlag modulierter Intensitätsanteil: $\Iac$
  
- "Untergrund"-Intensität: $\Idc$

- $\frac{\Iac}{\Idc} \approx \frac{\Delta I}{I_0} \propto - \color{blue}{\mu^A_a}\left(\color{green}{\spo}\right) \color{red}{\Delta x_A}$


#### R Faktor (*ratio of ratio*)

<div class="alert alert-block alert-info">

$$
\color{green}{\ros}
     =  \frac{ \nicefrac{I^{RED}_{AC}}{I^{RED}_{DC}} }{ \nicefrac{I^{IR}_{AC}}{I^{IR}_{DC}} } 
$$

</div class="alert alert-block alert-info">

***

$$
\quad\quad     {\;\approx\;} \color{blue}{
     \frac{\mux{A}{RED}\left(\color{green}{\spo}\right)}
          {\mux{A}{IR}\left(\color{green}{\spo}\right)}
          }
$$

[//]: # ( Zusammenhang mit Sauerstoffsättigung )



<div class="alert alert-block alert-info">
#### Aus den Verhältnissen der *Puls-modulierten* 
#### zu *nicht-modulierten Intensitäten* von Ergebnissen 
#### bei 2 verschiedenen Wellenlängen, $\color{#C82C1C}{RED}$ und $\color{#A38C89}{IR}$, 
#### wird der Quotient $\color{green}{\ros}$ gebildet. Dieser Wert ist
#### näherungsweise nur noch abhängig von $\color{green}{\sao}$.
</div class="alert alert-block alert-info">

***

<div class="alert alert-block alert-info">
$$
\begin{align}
\color{green}{\spo}
  &= 
  {\Tiny
     \frac{\color{green}{\ros}\mux{\hb}{IR}-\mux{\hb}{RED}}
          {\left(\mux{\hbo}{RED}-\mux{\hb}{RED}\right) 
           + \color{green}{\ros} \left(\mux{\hb}{IR} - \mux{\hbo}{IR}\right)} 
   } \\ 
\\                       
  &{\;\approx\;} \color{purple}{c_1} \color{green}{\ros} + \color{purple}{c_2}
\end{align}
$$
</div class="alert alert-block alert-info">

##### Achtung $\textsf{--}$ Streuung!

<img src="figs/basics/KanII-Fig5.9-reflectance_mode-scattering.png"  style="width: 40%" />

Das Bild zeigt einen Sensor in Reflexionsanordnung.

## Die Messung klappt?

***
### Demonstrator und Live-Experiment

### Live Experiment mit "Kalibrierung"

<div class="alert alert-block alert-success">

#### Versuch
$\ros$-Werten des Demonstrators im Vergleich zu <br>
$\spo$-Werte des kommerziellen Pulsoxymeters  <br>
mehrerer Probanden sollen für eine Kalibrierung des Sensors ermittelt werden.

- Werte tabellieren 
- Ausgleichsgerade bzw. lineare Regression <br>
  $\spo = c_1 \ros + c_2$

</div class="alert alert-block alert-success">

### Demonstrator Hardware

###### Sensormodul mit 3 LEDs und 1 Photodiode
<img src="figs/hardware/MAX30105_Close_up.jpg"  style="width: 40%" />

###### Anordnung auf Steckbrett
<img src="figs/hardware/ESP32_MAX30105_bb.png"  style="width: 40%" />

* Interface-Platine mit Maxim MAX30105
  - diese Variante mit 3 LEDs <br /> ($\lambda$: 527 nm, 660 nm, 880 nm) <br />
    ist offiziell ein "integriertes Partikelsensormodul für Rauchmeldeanlagen"  
  - MAX30102 mit 2 LEDs: "Pulsoximeter- & Herzfrequenz-Sensor"
  - I$^2$C Kommunikation ("2-Draht-Bus")

###### Funktionsschema Sensormodul
<img src="figs/hardware/Maxim_MAX30105_funcdia.png"  style="width: 80%" />

* Microcontroller Board (Sparkfun "ESP32 Thing") <br />mit Telemetrie über WLAN
* PC 
  - MQTT (Message Queuing Telemetry Transport) Server
  - Auswertesoftware (Python)

##### Kennwerte des Sensors beim Experiment:

| Parameter  | Wert       |
|:-----------|------------|
| LED Strom  | $\sim$4 mA |
| ADC Integrationszeit und LED-Pulsbreite (ca.)  | 411 $\mu$s  |
| LED Flankenabstand (RED $\leftrightarrow$ IR)  | 1107 $\mu$s |
| Mittelung und Datendezimierung   | 4 Samples   |
| übertragene Sampling-Rate        | $\sim$20 Hz |

<div class="alert alert-block alert-success">

#### Was sind die Ergebnisse ihres Versuches?

***

#### Warum ist unser Kalibrierversuch *nicht* optimal?

</div class="alert alert-block alert-success">

#### Vertiefende Literatur: Messtechnische Grundlagen und Auswertung
1. Eugenijus Kaniusas - *Biomedical Signals and Sensors* (insb. Band II)
1. John G. Webster - *Design of pulse oximeters*

## Was haben wir, und welche Punkte sind noch offen?

***
### Resümee und Ausblick

<div class="alert alert-block alert-success">
Was haben Sie gelernt?
</div class="alert alert-block alert-success">

###### Kenntnisse

- Anwendungsfelder
- reale Systeme und Messdaten
- Verständnis des Messprinzips und <br>
  der Berechnung $\spo$

###### Ausblick

- Laborexperiment: <br>
  Versuch inklusive Abtastung der Analogsignale
  
- Seminarthema: <br>
  Optische Parameter von Gewebe und Blut

<div class="alert alert-block alert-info">
<img src="figs/outlook/Danke_für_den_Input-Beule.png"  style="width: 60%" />
</div class="alert alert-block alert-info">

###### Lernfragen

- Was sind die Komponenten eines Pulsoxymeters?
- Was sind die entscheidenden "Tricks" der Messauswertung?
- Welche Messeinflüsse und -unsicherheiten sind zu beachten?

In [17]:
display(HTML('<img src="figs/eval/eval_oxy-steps_1to3.png"  style="width: 80%" />'))

FIR-Filter als (*very*) Low-Pass-Filter
<img src="figs/eval/filter-nearly_dc_low_pass.png"  style="width: 40%" />